In [1]:
%%time
# loading necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import re

CPU times: user 667 ms, sys: 252 ms, total: 919 ms
Wall time: 1.56 s


In [11]:
file_name='/content/en2cn-2k.en2nen2cn'
file = open(file_name,'r',encoding="utf8")
text = file.read()
# Removing last instance after splitting as it is empty string
text = text.split('\n')[:-1]
# Creating the pandas dataframe
data = [[text[i], text[i+1]] for i in list(range(0, 6000, 3))]
dataframe = pd.DataFrame(data, columns = ['informal_text', 'normal_text'])

## Inference
---   We can see that all the 3rd lines starting from 0 like 0,3,6,9 are informal text

---   The very next line of list after informal text is normal english text 

In [12]:
dataframe.head()

,informal_text,normal_text
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


### Augmentation

In [4]:
pip install numpy requests nlpaug

     |████████████████████████████████| 405 kB 3.3 MB/s 


In [18]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [5]:
!wget --header="Host: dl.fbaipublicfiles.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://fasttext.cc/" "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip" -c -O 'wiki-news-300d-1M.vec.zip'

--2021-09-07 19:36:26--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  20.5MB/s    in 32s     

2021-09-07 19:36:59 (20.1 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [6]:
!unzip ./wiki-news-300d-1M.vec.zip

Archive:  ./wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [13]:
import nlpaug.augmenter.word as naw
from tqdm import tqdm
#Reference is taken from https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb
def spelling_augmentation(data):
    normal_text = list(data["normal_text"].values)
    aug_text = []
    aug = naw.SpellingAug()
    for sentence in tqdm(normal_text):
        aug_text.append(aug.augment(sentence))    
    return normal_text, aug_text

In [14]:
#Reference is taken from https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb
def word_augmentation(data):
    normal_text = list(data["normal_text"].values)
    aug_text = []
    aug = naw.WordEmbsAug(model_type='fasttext', model_path='/content/wiki-news-300d-1M.vec',action="substitute")
    for sentence in tqdm(normal_text):
        aug_text.append(aug.augment(sentence)) 
    return  normal_text,aug_text

In [15]:
#Reference is taken from https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb
def syn_augmentation(data):
    normal_text = list(data["normal_text"].values)
    aug_text = []
    aug = naw.SynonymAug()
    for sentence in tqdm(normal_text):
        aug_text.append(aug.augment(sentence))    
    return normal_text, aug_text

In [16]:
normal_text, aug_text=spelling_augmentation(dataframe)
df_1 = pd.DataFrame({"informal_text":aug_text,"normal_text":normal_text})

normal_text, aug_text=word_augmentation(dataframe)
df_2 = pd.DataFrame({"informal_text":aug_text,"normal_text":normal_text})

data1 = pd.concat([df_1,df_2])

100%|██████████| 2000/2000 [39:08<00:00,  1.17s/it]


In [19]:
data2=pd.concat([dataframe,data1])
normal_text, aug_text=syn_augmentation(data2)
df_3 = pd.DataFrame({"informal_text":aug_text,"normal_text":normal_text}) 

augmented_data = pd.concat([df_3,data2])
augmented_data.reset_index(inplace=True,drop=True)

100%|██████████| 6000/6000 [00:13<00:00, 460.47it/s]


In [20]:
augmented_data.shape

(12000, 2)

In [21]:
augmented_data.iloc[[0,2000,4000,6000,8000,10000,11999]]

,informal_text,normal_text
0,Do you want me to hold seat for you operating ...,Do you want me to reserve seat for you or not?
2000,Do you require me to reserve backside for you ...,Do you want me to reserve seat for you or not?
4000,Do you want me to appropriate seat for you or ...,Do you want me to reserve seat for you or not?
6000,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
8000,Do wou want be to reserve seat for you ou not?,Do you want me to reserve seat for you or not?
10000,Do you chose me to reserve votes for you there...,Do you want me to reserve seat for you or not?
11999,I bring for you. I can completely promiser you...,I bring for you. I can not promise you 100% to...


In [22]:
augmented_data['informal_text_inp'] = '<' + augmented_data['informal_text'].astype(str)+ '>'
augmented_data['normal_text_inp'] = '<' + augmented_data['normal_text'].astype(str)
augmented_data['normal_text_out'] = augmented_data['normal_text'].astype(str) + '>'

In [23]:
augmented_data = augmented_data.drop(['normal_text','informal_text'], axis=1)
augmented_data.head()

,informal_text_inp,normal_text_inp,normal_text_out
0,<Do you want me to hold seat for you operating...,<Do you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?>
1,<Yeap. You reaching? We ordered some Durian pa...,<Yeap. You reaching? We ordered some Durian pa...,Yeap. You reaching? We ordered some Durian pas...
2,<They turn more expensive already. Mine be com...,<They become more expensive already. Mine is l...,They become more expensive already. Mine is li...
3,<1 ' m Thai. What do you make?>,<I'm Thai. What do you do?,I'm Thai. What do you do?>
4,<How do you do! How did your week go? Haven ' ...,<Hi! How did your week go? Haven't heard from ...,Hi! How did your week go? Haven't heard from y...


In [24]:
augmented_data.shape

(12000, 3)

In [25]:
# Filtering out sentences of length more than 200 for normal and 160 for informal which we found after doing EDA
augmented_data = augmented_data[augmented_data['informal_text_inp'].str.split().apply(len) <= 160]
augmented_data = augmented_data[augmented_data['normal_text_inp'].str.split().apply(len) <= 200]
augmented_data = augmented_data[augmented_data['normal_text_out'].str.split().apply(len) <= 200]
augmented_data.head()

,informal_text_inp,normal_text_inp,normal_text_out
0,<Do you want me to hold seat for you operating...,<Do you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?>
1,<Yeap. You reaching? We ordered some Durian pa...,<Yeap. You reaching? We ordered some Durian pa...,Yeap. You reaching? We ordered some Durian pas...
2,<They turn more expensive already. Mine be com...,<They become more expensive already. Mine is l...,They become more expensive already. Mine is li...
3,<1 ' m Thai. What do you make?>,<I'm Thai. What do you do?,I'm Thai. What do you do?>
4,<How do you do! How did your week go? Haven ' ...,<Hi! How did your week go? Haven't heard from ...,Hi! How did your week go? Haven't heard from y...


In [26]:
augmented_data.shape

(12000, 3)

In [27]:
"""Saving the data into the csv file for further steps"""
augmented_data.to_csv("with_augmentation_2.csv",index=False)